<a href="https://colab.research.google.com/github/iulihardt/DataScience/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf

import tensorflow_datasets as tfds

TensorFlow 2.x selected.


In [0]:
mnist_dataset, mnist_info = tfds.load(name="mnist", as_supervised=True, with_info=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 2000

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


In [3]:
mnist_dataset['test']

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

## Model

In [0]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #quantas layers quiser
                             tf.keras.layers.Dense(150, activation='relu'),
                             tf.keras.layers.Dense(100, activation='relu'),
                             tf.keras.layers.Dense(50, activation='relu'),
                             tf.keras.layers.Dense(100, activation='relu'),
                             tf.keras.layers.Dense(150, activation='relu'),
                             tf.keras.layers.Dense(200, activation='relu'),
                             tf.keras.layers.Dense(100, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
                             ])



In [0]:

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
NUM_EPOCHS = 10
NUM_STEPS = num_validation_samples#//BATCH_SIZE



model.fit(train_data,epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2, validation_steps=NUM_STEPS)

Epoch 1/10
27/27 - 27s - loss: 1.3125 - accuracy: 0.5721 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
27/27 - 25s - loss: 0.3793 - accuracy: 0.8881 - val_loss: 0.2874 - val_accuracy: 0.9205
Epoch 3/10
27/27 - 25s - loss: 0.2317 - accuracy: 0.9342 - val_loss: 0.2107 - val_accuracy: 0.9413
Epoch 4/10
27/27 - 25s - loss: 0.1732 - accuracy: 0.9492 - val_loss: 0.1710 - val_accuracy: 0.9507
Epoch 5/10
27/27 - 24s - loss: 0.1415 - accuracy: 0.9589 - val_loss: 0.1395 - val_accuracy: 0.9602
Epoch 6/10
27/27 - 25s - loss: 0.1145 - accuracy: 0.9661 - val_loss: 0.1200 - val_accuracy: 0.9677
Epoch 7/10
27/27 - 26s - loss: 0.0976 - accuracy: 0.9716 - val_loss: 0.1038 - val_accuracy: 0.9710
Epoch 8/10
27/27 - 25s - loss: 0.0833 - accuracy: 0.9752 - val_loss: 0.0868 - val_accuracy: 0.9767
Epoch 9/10
27/27 - 24s - loss: 0.0718 - accuracy: 0.9787 - val_loss: 0.0870 - val_accuracy: 0.9772
Epoch 10/10
27/27 - 23s - loss: 0.0661 - accuracy: 0.9806 - val_loss: 0.0773 - val_accuracy: 0.9783


In [7]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 2s 2s/step - loss: 0.1077 - accuracy: 0.9696
